# MODELOS

IMPORTACION DE LIBRERIAS

In [1]:
# Bibliotecas estándar
import os
import joblib
import concurrent.futures
import re
import random
from collections import Counter
from datetime import datetime

# Bibliotecas científicas
import numpy as np

# Bibliotecas de análisis y modelado de datos
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.model_selection import TimeSeriesSplit

# Librerias de modelos
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor, XGBClassifier
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Bibliotecas de visualización
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler


c:\Users\joant\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CARGA DE DATOS

In [2]:
df = pd.read_csv(r'.\..\..\Datos_Proyecto\observations_full.csv')

df['date'] = pd.to_datetime(df['date'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

SVM

In [14]:
# Select features and target
features = ['year', 'month', 'day', 'precipitation', 'wind', 'humidity', 'visibility', 'estacion_id']
target = 'weather_id'

# Split data into train and test sets
X = df[features]
y = df[target]

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)

# Scale the data (using StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SVC(probability=True, C=0.1, gamma='scale', kernel='rbf', class_weight='balanced')
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# ROC AUC (for multi-class)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test_scaled), multi_class='ovr')
print(f"ROC AUC: {roc_auc}")

# Save the model using joblib
joblib.dump(model, r'.\..\Modelos\SVM_weather_id.pkl')

Accuracy: 0.8421385637456501
Classification Report:
              precision    recall  f1-score   support

           1       0.84      0.87      0.86      1897
           2       0.83      0.84      0.84      1896
           3       0.75      0.57      0.65      1897
           4       0.86      0.97      0.91      1897
           5       0.89      0.97      0.93      1896

    accuracy                           0.84      9483
   macro avg       0.84      0.84      0.84      9483
weighted avg       0.84      0.84      0.84      9483

Confusion Matrix:
[[1643    5  156   93    0]
 [  18 1587  197   94    0]
 [ 264  280 1078   71  204]
 [  21   32    0 1831   13]
 [   0    0    0   49 1847]]
ROC AUC: 0.9690939074866127


['.\\..\\Modelos\\SVM_weather_id.pkl']

XGBoosting Classifier

In [13]:
# Seleccionar las características y el objetivo
features = ['year', 'month', 'day', 'precipitation', 'wind', 'humidity', 'visibility', 'estacion_id']
target = 'weather_id'

# Dividir los datos en características (X) y objetivo (y)
X = df[features]
y = df[target]

# Ajustar las etiquetas para que comiencen desde 0
y_adjusted = y - 1  # Ahora las clases estarán en el rango [0, 1, 2, 3, 4]

# Mostrar la distribución original de las clases
print("Distribución original de las clases:", Counter(y_adjusted))

# Crear una instancia de SMOTE
smote = SMOTE(random_state=42, k_neighbors=10)
X_resampled, y_resampled = smote.fit_resample(X, y_adjusted)

# Mostrar la nueva distribución de las clases
print("Distribución de clases después de SMOTE:", Counter(y_resampled))

# Dividir los datos balanceados en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Crear un modelo de XGBoost
model = XGBClassifier(random_state=42, n_estimators=300, max_depth=30)

# Ajustar el modelo con los datos de entrenamiento balanceados
model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Reconvertir las predicciones y las etiquetas originales
y_pred = y_pred + 1  # Volver a [1, 2, 3, 4, 5]
y_test = y_test + 1  # Volver a [1, 2, 3, 4, 5]

# Calcular la exactitud (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Evaluar el rendimiento del modelo
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar la matriz de confusión
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))

# Guardar el modelo entrenado usando joblib
joblib.dump(model, r'.\..\Modelos\XGB_weather_id.pkl')

Distribución original de las clases: Counter({1: 9483, 0: 9024, 2: 5959, 3: 429, 4: 105})
Distribución de clases después de SMOTE: Counter({0: 9483, 2: 9483, 1: 9483, 3: 9483, 4: 9483})
Accuracy: 0.9333544237055784

Reporte de clasificación:
              precision    recall  f1-score   support

           1       0.89      0.97      0.93      1897
           2       0.89      0.95      0.92      1910
           3       0.93      0.74      0.82      1861
           4       0.98      1.00      0.99      1892
           5       0.99      1.00      0.99      1923

    accuracy                           0.93      9483
   macro avg       0.93      0.93      0.93      9483
weighted avg       0.93      0.93      0.93      9483


Matriz de confusión:
[[1840    1   38   18    0]
 [   0 1820   70   20    0]
 [ 229  225 1382    0   25]
 [   2    4    0 1886    0]
 [   0    0    0    0 1923]]


['.\\..\\Modelos\\XGB_weather_id.pkl']

RANDOM FOREST CLASSIFIER

In [12]:
# Seleccionar las características y el objetivo
features = ['year', 'month', 'day', 'precipitation', 'wind', 'humidity', 'visibility', 'estacion_id']
target = 'weather_id'

# Dividir los datos en características (X) y objetivo (y)
X = df[features]
y = df[target]

# Ajustar las etiquetas para que comiencen desde 0
y_adjusted = y - 1  # Ahora las clases estarán en el rango [0, 1, 2, 3, 4]

# Mostrar la distribución original de las clases
print("Distribución original de las clases:", Counter(y_adjusted))

# Crear una instancia de SMOTE
smote = SMOTE(random_state=42,k_neighbors=10)
X_resampled, y_resampled = smote.fit_resample(X, y_adjusted)

# Mostrar la nueva distribución de las clases
print("Distribución de clases después de SMOTE:", Counter(y_resampled))

# Dividir los datos balanceados en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Crear un modelo de Random Forest (puedes usar cualquier otro clasificador)
model = RandomForestClassifier(random_state=42, n_estimators=300, max_depth=30)

# Ajustar el modelo con los datos de entrenamiento balanceados
model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Reconvertir las predicciones y las etiquetas originales
y_pred = y_pred + 1  # Volver a [1, 2, 3, 4, 5]
y_test = y_test + 1  # Volver a [1, 2, 3, 4, 5]

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Evaluar el rendimiento del modelo
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar la matriz de confusión
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))

# Guardar el modelo entrenado usando joblib
joblib.dump(model, r'.\..\Modelos\RFC_weather_id.pkl')

Distribución original de las clases: Counter({1: 9483, 0: 9024, 2: 5959, 3: 429, 4: 105})
Distribución de clases después de SMOTE: Counter({0: 9483, 2: 9483, 1: 9483, 3: 9483, 4: 9483})
Accuracy: 0.9310344827586207

Reporte de clasificación:
              precision    recall  f1-score   support

           1       0.87      0.99      0.92      1897
           2       0.87      0.98      0.92      1910
           3       0.99      0.69      0.81      1861
           4       0.97      1.00      0.98      1892
           5       0.99      1.00      0.99      1923

    accuracy                           0.93      9483
   macro avg       0.94      0.93      0.93      9483
weighted avg       0.94      0.93      0.93      9483


Matriz de confusión:
[[1871    0    1   25    0]
 [   0 1866   11   33    0]
 [ 285  270 1283    0   23]
 [   1    1    1 1887    2]
 [   0    0    1    0 1922]]


['.\\..\\Modelos\\RFC_weather_id.pkl']

RNN

In [ ]:
# 1. Importar las bibliotecas necesarias
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 3. Seleccionar características y objetivo
features = ['year', 'month', 'day', 'precipitation', 'wind', 'humidity', 'visibility', 'estacion_id']
target = 'weather_id'

# Dividir en X (características) y y (objetivo)
X = df[features]
y = df[target]

# 4. Preprocesamiento de datos
# Normalizar las características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Codificar el objetivo (y) si es una clasificación categórica
y_encoded = to_categorical(y)  # Usamos to_categorical para convertir en "one-hot encoding"

# 5. Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Más neuronas
model.add(Dense(64, activation='relu'))  # Capa adicional
model.add(Dense(32, activation='relu'))  # Capa adicional
model.add(Dense(y_encoded.shape[1], activation='softmax'))  # Capa de salida

# 7. Compilar el modelo
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',  # Para clasificación multiclase
              metrics=['accuracy'])

# 8. Entrenar el modelo
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20,  # Número de épocas
                    batch_size=64)

# 9. Evaluar el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Pérdida en prueba: {test_loss}')
print(f'Precisión en prueba: {test_accuracy}')

# 10. Hacer predicciones
predictions = model.predict(X_test)

# Opcional: Decodificar las predicciones de vuelta a sus clases originales
predicted_classes = predictions.argmax(axis=1)

# Decodificar las etiquetas reales (y_test) si están en formato one-hot
y_test_classes = y_test.argmax(axis=1)

# 11. Calcular y mostrar la matriz de confusión
cm = confusion_matrix(y_test_classes, predicted_classes)
print(cm)

model.save(r'.\..\Modelos\RNN_weather_id.h5')

Epoch 1/20


c:\Users\joant\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7159 - loss: 0.8051 - val_accuracy: 0.8390 - val_loss: 0.4546
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8495 - loss: 0.4348 - val_accuracy: 0.8716 - val_loss: 0.3938
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8687 - loss: 0.3872 - val_accuracy: 0.8714 - val_loss: 0.3734
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8799 - loss: 0.3528 - val_accuracy: 0.8810 - val_loss: 0.3573
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8813 - loss: 0.3500 - val_accuracy: 0.8834 - val_loss: 0.3415
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8833 - loss: 0.3350 - val_accuracy: 0.8838 - val_loss: 0.3415
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8845 - loss: 0.3317 - val_accuracy: 0.8830 - val_loss: 0.3368
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8858 - loss: 0.3288 - val_accuracy: 0.8848 - val_

# PREDICCIONES

Mejor modelo para weather_id: **Random Forest Classifier**

In [ ]:
# Cargar el DataFrame y convertir la columna de fecha
predictions = pd.read_csv(r'.\..\weather_predictions.csv')
predictions['date'] = pd.to_datetime(predictions['date'])

# Extraer año, mes y día de la fecha
predictions['year'] = predictions['date'].dt.year
predictions['month'] = predictions['date'].dt.month
predictions['day'] = predictions['date'].dt.day

# Cargar el modelo
modelo_RFC = joblib.load(r'.\..\Modelos\RandomForest\Classifier\weather_id_RFC.pkl')

# Seleccionar las características requeridas para el modelo
features = [
    'year', 'month', 'day', 'precipitation', 'temp_max', 'temp_min',
    'wind', 'humidity', 'pressure', 'solar_radiation', 'visibility', 'cloudiness_id'
]

# Crear la matriz de características para el modelo
X = predictions[features]

# Generar predicciones con el modelo
predictions['weather_id'] = modelo_RFC.predict(X).round(0).astype(int)

# Guardar el DataFrame actualizado
predictions.to_csv(r'.\..\weather_predictions.csv', index=False)
